In [ ]:
import lucene
lucene.initVM()

In [ ]:
lucene.VERSION

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [ ]:
bad_words = pd.read_csv('/Project/data/bad_word.txt', header=None)
bad_words_set = set(bad_words[0].values)

In [ ]:
# jigsaw_dataset = pd.read_csv('data/jigsaw_preprocessed_labels.csv',header=0)
# jigsaw_labels = jigsaw_dataset['label']

In [ ]:
news_dataset = pd.read_csv('/Project/data/pos_tags_dataset.csv',header=0)
# labels = pd.read_csv('data/comments_model1_outputs.txt',header=None)

In [ ]:
news_dataset['words']

In [ ]:
rows_to_ignore = pd.read_csv('/Project/data/rows_to_ignore.txt',header=None)

In [ ]:
news_dataset = news_dataset.drop(news_dataset.index[rows_to_ignore[0].unique()])

In [ ]:
labels= []
word_indices=[]
for sentence in news_dataset['cleaned_comments']:    
    #capture bw indices for each sentence
    flag = False
    try:
        for word in sentence.split():
            if word.lower() in bad_words_set:
                flag = True
        labels.append(1) if flag else labels.append(0)
    except:
        labels.append(0)

In [82]:
news_dataset['label'] = labels

In [89]:
# news_dataset.to_csv('/Project/data/pos_tags_labels_dataset.csv',index=False)

In [85]:
bad_sentences = news_dataset[news_dataset['label']==1]
good_sentences = news_dataset[news_dataset['label']==0]

In [88]:
bad_sent = bad_sentences[['cleaned_comments','label', 'pos_tag']]
good_sent = good_sentences[['cleaned_comments','label', 'pos_tag']]

In [90]:
bad_comments = []
good_comments = []
for bad_rows in bad_sent.itertuples(index = True):
    bad_comments.append(bad_rows[1])

for good_rows in good_sent.itertuples(index = True):
    good_comments.append(good_rows[1])


In [91]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.document import Document, Field, StringField, TextField
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, DirectoryReader
from org.apache.lucene.search import IndexSearcher,ScoreDoc, TopDocs
from org.apache.lucene.queries.mlt import MoreLikeThisQuery
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import Version
import java
import os

# Indexing
index_dir = '/Project/data/indexdir'
# analyzer = StandardAnalyzer()
analyzer = EnglishAnalyzer()

directory = FSDirectory.open(java.nio.file.Paths.get(index_dir))
config = IndexWriterConfig(analyzer)
writer = IndexWriter(directory, config)

if not os.path.exists(index_dir):
    print('Created dir')
    os.mkdir(index_dir)
    doc1 = Document()
    doc1.add(Field("content", "The quick brown fox jumped over the lazy dog",  TextField.TYPE_STORED))
    writer.addDocument(doc1)
    
else:
    # Insert all good sentences to segments
    for good_sentence in good_sentences['cleaned_comments']:
        doc = Document()
        doc.add(TextField("content", good_sentence, Field.Store.YES))
        writer.addDocument(doc)

# writer.commit()
writer.close()


In [92]:
from tqdm import tqdm
# Searching
searcher = IndexSearcher(DirectoryReader.open(directory))
analyzer = EnglishAnalyzer()
final_similar_sentences = []
# Get MoreLikeThisQuery for the first document
# more_like_this = MoreLikeThisQuery("dude my employer uses svb we are dead in the water right now",["content"],analyzer,'content')
for bad_sentence in tqdm(bad_sentences['cleaned_comments']):
    # print(f' BS: {bad_sentence}')
    similar_sentences=[]
    more_like_this = MoreLikeThisQuery(bad_sentence,["content"],analyzer,"content")
    hits = searcher.search(more_like_this,10).scoreDocs
    # print results
    # print(f"Found {len(hits)} hits:")
    for hit in hits:
        doc = searcher.doc(hit.doc)
        # print(f"{doc['content']}   score: {hit.score}")
        similar_sentences.append(doc['content'])
    final_similar_sentences.append(similar_sentences)

100%|██████████| 24051/24051 [00:22<00:00, 1060.69it/s]


In [102]:
final_similar_sentences[954]

['can cancers get cancers ',
 'those responsible for the killings have been sacked ',
 'yes i have two cancers at hte same time',
 ' bans entry syria security agent killings video ',
 'us is sitting at around reported deaths so for we racked up about deaths by covid tracking in was slow but only showing about deaths i personally think it is much underreported was about deaths for the us ',
 'care home deaths ',
 'were the two guns that were used in the killings ever recovered ',
 'within cells interlinked ',
 ' to make fun of the cell phone guy',
 'do you really need more rosd deaths']

In [101]:
bad_sentences['cleaned_comments'].iloc[954]

'that does not describe the situation right at all it is primarily due to inflammation forcing more cell death and division and all three because it obesity fucks with other things too like hormone production signaling to the body in other ways to divide cells more smoking and alcohol also because direct cell death alcohol is used to denature and kill and we drink it every cell killed as you swallow it down into your mouth down your throat through your stomach intestines and so on not even including where else the body takes some of it through your blood even it is increasing the odds of cancer everywhere it passes over we are not constantly battling cancer it is not lying in wait to pounce or constantly assaulting us it is more like we are unknowingly doing things that may let a trojan horse of former cells now undying cells loose from somewhere '